# Process Scopus title (e.g. journal) metadata

In [1]:
import pathlib
import re

import pandas
import slugify

from utilities import format_issn

In [2]:
# Load dataset
path = pathlib.Path('download/Scopus_Source_List.xlsx')

In [3]:
source_df = pandas.read_excel(path, sheet_name='Scopus Sources October 2017')
source_df.head(2)

,Sourcerecord id,Source Title (Medline-sourced journals are indicated in Green) Titles indicated in bold red do not meet the Scopus quality criteria anymore and therefore Scopus discontinued the forward capturing,Print-ISSN,E-ISSN,Active or Inactive,Coverage,Article language in source (three-letter ISO language codes),2014 CiteScore,2014 SJR,2014 SNIP,...,2700 Medicine,2800 Neuroscience,2900 Nursing,"3000 Pharmacology, Toxicology and Pharmaceutics",3100 Physics and Astronomy,3200 Psychology,3300 Social Sciences,3400 Veterinary,3500 Dentistry,3600 Health Professions
0,18500162600,21st Century Music,15343219,NaN,Inactive,2002-2011,ENG,NaN,0.111,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
1,21100404576,2D Materials,NaN,20531583,Active,2014-ongoing,ENG,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,Physics and Astronomy,NaN,NaN,NaN,NaN,NaN


In [4]:
conf_96_df = pandas.read_excel(path, sheet_name='Conf. Proceedings post-1995')
conf_96_df.head(2)

,Sourcerecord id,Source title,ISSN,Coverage,2014 CiteScore,2014 SJR,2014 SNIP,2015 Citescore,2015 SJR,2015 SNIP,2016 CiteScore,2016 SJR,2016 SNIP,ASJC code
0,21100496635,100th Annual Conference and Exhibition of the ...,NaN,2007,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,2105; 2102
1,21100496732,100th Annual Conference and Exhibition of the ...,NaN,2007,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,1709; 1705


In [5]:
conf_95_df = pandas.read_excel(path, sheet_name='Conf. Proceedings pre-1996')
conf_95_df.head(2)

,Sourcerecord id,Source title,Coverage,ISSN,ASJC code
0,130776,"10th ASM Conf on Adv in the Prod of Tubes, Bar...",1987,NaN,2200;
1,130669,10th ASM Conference on Advances in the Product...,1987,NaN,2200;


In [6]:
code_df = pandas.read_excel(path, sheet_name='ASJC classification codes', usecols=1).dropna()
code_df.Code = code_df.Code.astype(int)
code_df.head(2)

,Code,Description
1,1000,Multidisciplinary
3,1100,General Agricultural and Biological Sciences


In [7]:
# Rename columns on dataframes
renamer = {
    'Sourcerecord id ': 'scopus_id',
    'All Science Classification Codes (ASJC)': 'asjc_codes',
    'ASJC code': 'asjc_codes',
    'Print-ISSN': 'issn_print',
    'E-ISSN': 'issn_electronic',
    'ISSN': 'issn_unspecified',
}

for df in source_df, conf_96_df, conf_95_df:
    df.rename(columns=renamer, inplace=True)

## Titles

In [8]:
# Extract title names
rows = list()
for df in source_df, conf_96_df, conf_95_df:
    rows.extend(zip(df.scopus_id, df.iloc[:, 1]))

rows.sort()
title_df = pandas.DataFrame(rows, columns=['scopus_id', 'title_name'])
title_df = title_df.drop_duplicates('scopus_id')

In [9]:
title_df.head(2)

,scopus_id,title_name
0,12000,Journal of Technology in Counseling
1,12001,Journal of the Experimental Analysis of Behavior


In [10]:
len(title_df)

72502

In [11]:
path = pathlib.Path('data/titles.tsv')
title_df.to_csv(path, index=False, sep='\t')

## ISSN

In [12]:
value_vars = {'issn_print', 'issn_electronic', 'issn_unspecified'}
issn_dfs = list()
for df in source_df, conf_96_df, conf_95_df:
    df_value_vars = value_vars.intersection(df.columns)
    df = pandas.melt(df, id_vars='scopus_id', value_vars=df_value_vars, var_name='issn_type', value_name='issn')
    df.dropna(inplace=True)
    df.issn = df.issn.map(format_issn)
    df.issn_type = df.issn_type.map(lambda x: x.split('_')[1])
    issn_dfs.append(df.dropna())
issn_df = (
    pandas.concat(issn_dfs)
    .dropna()
    .sort_values(['scopus_id', 'issn_type', 'issn'])
    .drop_duplicates(['scopus_id', 'issn'])
)

In [13]:
len(issn_df)

47348

In [14]:
issn_df.head(3)

,scopus_id,issn_type,issn
58544,12000,print,1527-6228
58763,12001,print,0022-5002
21966,12002,electronic,1520-6696


In [15]:
path = pathlib.Path('data/issn.tsv')
issn_df.to_csv(path, sep='\t', index=False)

## ASJC Codes

All Science Journal Classification (ASJC) Codes

In [16]:
# Extract ASJC code descriptions
code_df = code_df.rename(columns={'Code': 'asjc_code', 'Description': 'asjc_description'})
code_df = code_df[['asjc_code', 'asjc_description']]
code_df.head(2)

,asjc_code,asjc_description
1,1000,Multidisciplinary
3,1100,General Agricultural and Biological Sciences


In [17]:
path = pathlib.Path('data/asjc-codes.tsv')
code_df.to_csv(path, index=False, sep='\t')

In [18]:
# Extract title to code mapping
rows = list()
for df in source_df, conf_96_df, conf_95_df:
    for scopus_id, codes in zip(df.scopus_id, df.asjc_codes):
        for code in re.split(r'[,;] *', str(codes)):
            code = code.strip()
            try:
                code = int(code)
            except ValueError:
                continue
            rows.append((scopus_id, code))

rows.sort()
title_codes_df = pandas.DataFrame(rows, columns=['scopus_id', 'asjc_code'])
title_codes_df = title_codes_df.drop_duplicates()
title_codes_df.head(2)

,scopus_id,asjc_code
0,12000,1705
1,12000,3304


In [19]:
path = pathlib.Path('data/titles-asjc-codes.tsv')
title_codes_df.to_csv(path, index=False, sep='\t')

In [20]:
# Extract title to subject area mapping
subject_df = title_codes_df.copy()
subject_df.asjc_code = 100 * (subject_df.asjc_code // 100)
subject_df = subject_df.drop_duplicates()
subject_df = subject_df.merge(code_df)
subject_df.asjc_description = subject_df.asjc_description.map(lambda x: re.sub(r'^General ', '', x))
subject_df = subject_df.sort_values(['scopus_id', 'asjc_code'])
subject_df.head(2)

,scopus_id,asjc_code,asjc_description
0,12000,1700,Computer Science
18380,12000,3300,Social Sciences


In [21]:
path = pathlib.Path('data/subject-areas.tsv')
subject_df.to_csv(path, index=False, sep='\t')

## Title attributes

In [22]:
# Extract title attributes for non-conference-proceedings
renamer = {
    'Open Acces status, i.e., registered in DOAJ and/or ROAD. Status April 2017\n': 'open_access',
    'Active or Inactive': 'active',
    'Source Type': 'source_type',
    'Publisher imprints grouped to main Publisher': 'main_publisher',
    "Publisher's Country ": 'publisher_country',
}
attribute_df = source_df.copy()
attribute_df = attribute_df.rename(columns=renamer)
attribute_df = attribute_df[['scopus_id'] + list(renamer.values())]
attribute_df.active = (attribute_df.active == 'Active').astype(int)
attribute_df.open_access = attribute_df.open_access.notnull().astype(int)
attribute_df = attribute_df.sort_values('scopus_id')
# Enforce column order for diff viewing
columns = ['scopus_id', 'active', 'open_access', 'main_publisher', 'source_type', 'publisher_country']
attribute_df = attribute_df[columns]
attribute_df = attribute_df.drop_duplicates('scopus_id')
attribute_df.head(2)

,scopus_id,active,open_access,main_publisher,source_type,publisher_country
21712,12000,0,0,Columbus State University,Journal,United States
21931,12001,1,0,Wiley-Blackwell,Journal,United States


### Fix publishers

In [23]:
print(f'Publishers before filtering: {attribute_df.main_publisher.nunique():,}')

# Strip whitespace sourrounding publisher names
attribute_df.main_publisher = attribute_df.main_publisher.str.strip()
print(f'Publishers after whitespace stripping: {attribute_df.main_publisher.nunique():,}')

# Apply manual publisher name patches
path = pathlib.Path('data/publisher-name-patches.tsv')
patch_df = pandas.read_table(path)
publisher_renamer = dict(zip(patch_df.scopus_main_publisher, patch_df.publisher_name))
attribute_df.main_publisher.replace(publisher_renamer, inplace=True)
print(f'Publishers after renaming: {attribute_df.main_publisher.nunique():,}')

Publishers before filtering: 10,890
Publishers after whitespace stripping: 10,886
Publishers after renaming: 10,642


In [24]:
# Create a publsher dataframe
publisher_df = (
    attribute_df
    .groupby('main_publisher')
    .apply(len)
    .rename('n_journals')
    .reset_index()
)
slugger = slugify.Slugify(to_lower=True)
publisher_df['main_publisher_slug'] = publisher_df.main_publisher.map(slugger)

def summarize(df):
    """
    Condense multiple publishers with the same slug
    """
    # Select name variant used for the most number of titles
    df = df.sort_values(['n_journals', 'main_publisher'])
    return {
        'n_journals': sum(df.n_journals),
        'main_publisher': df.main_publisher.iloc[0],
        'name_variants': len(df),
    }

results = (
    publisher_df
    .groupby('main_publisher_slug')
    .apply(summarize)
)
publisher_df = pandas.DataFrame.from_records(results, index=results.index).reset_index()
publisher_df.head(2)

,main_publisher_slug,main_publisher,n_journals,name_variants
0,1105-media-inc,1105 Media Inc.,1,1
1,3g-publishing,3G Publishing,1,1


In [25]:
# How many publishers had name variants (identified using slug reduction)
publisher_df.name_variants.value_counts()

1    10543
2       48
3        1
Name: name_variants, dtype: int64

In [26]:
# Standardize name variants using slug reductionism
slug_to_publisher = dict(zip(publisher_df.main_publisher_slug, publisher_df.main_publisher))
attribute_df.main_publisher = attribute_df.main_publisher.map(slugger, na_action='ignore').map(slug_to_publisher)
print(f'Publishers after slug reductions: {attribute_df.main_publisher.nunique():,}')

Publishers after slug reductions: 10,592


In [27]:
path = pathlib.Path('data/title-attributes.tsv')
attribute_df.to_csv(path, index=False, sep='\t')

path = pathlib.Path('data/publishers.tsv')
publisher_df.to_csv(path, index=False, sep='\t')

In [28]:
# Top level subject areas
rows = []
tl_cols = list(source_df.columns[source_df.columns.str.startswith('Top level:')])
for i, series in source_df[['scopus_id'] + tl_cols].iterrows():
    scopus_id = series.scopus_id
    for value in series.iloc[1:].dropna():
        rows.append((scopus_id, value))
rows.sort()
top_df = pandas.DataFrame(rows, columns=['scopus_id', 'top_level_subject'])
top_df.head(2)

,scopus_id,top_level_subject
0,12000,Physical Sciences
1,12000,Social Sciences


In [29]:
path = pathlib.Path('data/title-top-levels.tsv')
top_df.to_csv(path, index=False, sep='\t')